In [ ]:
using GeoDataFrames
using DataFrames
using Plots
using Shapefile
using ArchGDAL
using GeoInterface
using ColorSchemes

# Set a backend for Plots.jl
gr()

In [ ]:

### Load Shapefiles
# Municipalities
shapefile_path_municipalities = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\MT_Municipios_2022\\MT_Municipios_2022.shp"
municipalities = Shapefile.Table(shapefile_path_municipalities) |> DataFrame

# Highways
shapefile_path_highways = "C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Análises\\Rodovias\\SNV_202410A.shp"
highways = Shapefile.Table(shapefile_path_highways) |> DataFrame

In [ ]:
### Filter Municipalities
# List of municipalities to filter
municipios_filtrados_nomes = [
    "Cáceres", "Primavera do Leste", "Rondonópolis", "Sinop", "Sorriso",
    "Guarantã do Norte", "Matupá", "Peixoto de Azevedo", "Terra Nova do Norte",
    "Nova Santa Helena", "Itaúba", "Lucas do Rio Verde", "Nova Mutum",
    "Barra do Garças", "Campo Verde"
]

# Filter the municipalities DataFrame
filtered_municipalities = filter(row -> row.NM_MUN in municipios_filtrados_nomes, municipalities)

In [ ]:
### Filter and Process Highways
# Filter for highways where sg_uf equals "MT" and vl_br is 070 or 163
filtered_highways_MT = filter(row -> row.sg_uf == "MT" && row.vl_br in ["070", "163"], highways)

# Group highways by the 'vl_br' column
grouped_highways = groupby(filtered_highways_MT, :vl_br)

In [ ]:
# Function to combine geometries within a group
function combine_geometries(geometries)
    # Filter out any missing or invalid geometries before processing
    valid_geoms = filter(g -> !isnothing(g) && GeoInterface.npoint(g) > 0, geometries)
    if isempty(valid_geoms)
        return nothing
    end

    return mapfoldl(
        x -> GeoInterface.convert(ArchGDAL, x), # map: convert each geometry to an ArchGDAL object
        ArchGDAL.union,                          # reduce: combine two geometries
        valid_geoms                              # the collection to operate on
    )
end

# Apply the function to create a new DataFrame with combined geometries
combined_highways = combine(grouped_highways, :geometry => combine_geometries => :geometry)

In [ ]:

### Plotting Section
# 1. Initialize the plot
plot(size=(1000, 800), dpi=1800)

# 2. Plot the base layer of all municipalities in MT
plot!(municipalities.geometry,
    color=:white,
    linecolor=:black,
    label=false
)

# 3. Define a specific color for each filtered municipality
# Using a dictionary to map city names to colors
municipality_color_map = Dict(
    "Barra do Garças" => :red,
    "Cáceres" => :royalblue,
    "Primavera do Leste" => :limegreen,
    "Rondonópolis" => :darkorchid,
    "Sinop" => :gold,
    "Sorriso" => :darkorange,
    "Guarantã do Norte" => :magenta,
    "Matupá" => :cyan,
    "Peixoto de Azevedo" => :saddlebrown,
    "Terra Nova do Norte" => :deeppink,
    "Nova Santa Helena" => :olivedrab,
    "Itaúba" => :darkgoldenrod,
    "Lucas do Rio Verde" => :slateblue,
    "Nova Mutum" => :mediumseagreen,
    "Campo Verde" => :tomato
)

# 4. Plot the highlighted municipalities using the color map
for row in eachrow(filtered_municipalities)
    city_name = row.NM_MUN
    plot!(row.geometry,
        color = municipality_color_map[city_name], # Look up color by city name
        label = city_name,
        linecolor = :darkgray
    )
end

# 5. Define specific colors for the highways (black and gray)
high_contrast_colors = [:black, :gray]

# 6. Loop through the combined highways and plot each one
for (i, row) in enumerate(eachrow(combined_highways))
    plot!(row.geometry,
        color = high_contrast_colors[i],
        linewidth = 4,
        label = "BR-" * string(row.vl_br)
    )
end

# 7. Finalize plot aesthetics
plot!(
    legend = :outerright,
    legendfontsize = 8,
     legendtitle = "Cities and Highways",
    framestyle = :none,
    grid = false
)

# 8. Display the plot
display(current())

# 9. Save the plot
Plots.savefig("C:\\Users\\daves\\OneDrive\\Pessoal\\Acadêmico\\Mestrado\\Dissertação - Execução\\Dissertação\\Quadros e gráficos\\Graph-Cities BR070 and BR163")